In [4]:
import pandas as pd
import shutil
from datetime import datetime

raw_data = pd.read_csv("raw_data.csv")
today = datetime.now()

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
driver = webdriver.Chrome()
# driver.get("https://www.loopnet.com/for-sale/il/?sk=7be1e58c3731f70583f9758ea8a63837")
property_data = []
loopnet_links = [
#     "https://www.loopnet.com/for-sale/commercial-land/?sk=d70f8d8955720de2a26a20c741090ea4&bb=nr3hw-hgpMuvu6ouiQ"
#     "https://www.loopnet.com/for-sale/commercial-land/{}/?sk=f547d78e0e5997281e090b5d69e7fd51&bb=zgk21_muoMlkz84joH"
#     "https://www.loopnet.com/for-sale/commercial-land/{}/?sk=5a4754c86dce9b8f968a72b3c86cf7e6&bb=o_g_0nkooM3ris74kH"
#     "https://www.loopnet.com/for-sale/ia/commercial-land/{}/?sk=0e29bff02e4ca8150d448c8055c149c4"
#     "https://www.loopnet.com/for-sale/land/{}/?sk=36b0f5fccc9a3cd9c997387d2c58518f"
    "https://www.loopnet.com/for-sale/land/{}/?sk=3744769fe81e64530e73ea160591ad63&bb=_8s2hg01_J-mh1i0_G"
#     "https://www.loopnet.com/for-sale/land/{}/?sk=28f233500acff4396e5c84a31007a840"
]

for loopnet_link in loopnet_links:
    driver = webdriver.Chrome()
    for x in range(2):
        driver.get(loopnet_link.format(x + 1))
        items = driver.find_elements_by_tag_name("article")
        for x in items:
            temp_data = []
            temp_sub_data = []
            temp_sf = []
            input_string = x.get_attribute("ng-click").split("'")[1].split("'")[0]
            temp_data.append(input_string)
            try:
                link = input_string.split("Listing/")[1].split("/")[0]
                if bool(re.search(r'\d', link)) and "upsell" not in link:
                    temp_data.append(" ".join(temp_data[0].split("Listing/")[1].split("/")[0].split("-")))
                    for index, val in enumerate(x.find_elements_by_tag_name("li")):
                        if index == 0:
                            if "$" in val.text:
                                price = [val.text]
                            else:
                                price = [None]
                        elif "SF" in val.text or "AC" in val.text:
                            temp_sf = [val.text]
                        elif val.text != "":
                            temp_sub_data.append(val.text)
                    temp_sf = [None] if not temp_sf else temp_sf
                    property_data.append(temp_data + price + temp_sf + [", ".join(temp_sub_data), today])
            except:
                continue

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=76.0.3809.87)


In [ ]:
import pandas as pd
today_df = pd.DataFrame(property_data, columns = ["Link", "Address", "Price", "Size", "Details", "Upload"])
raw_data = pd.concat([today_df, raw_data], ignore_index=True, sort=True)
raw_data = raw_data.drop_duplicates(subset="Link", keep='first', inplace=False)
raw_data = raw_data[["Link", "Address", "Price", "Size", "Details", "Upload"]]

In [ ]:
today_df.to_csv("Daily Import/{}.csv".format(today.strftime("%Y-%m-%d %H.%M%p")), index=False)
shutil.move("raw_data.csv", "Raw Data/{}.csv".format(today.strftime("%Y-%m-%d %H.%M%p")))
raw_data.to_csv("raw_data.csv", index=False)

In [ ]:
df = pd.read_csv("raw_data.csv")

target_cities = pd.read_csv("preferred_cities.csv")["City Name"].tolist()

# filtered_df = df[df["Address"].str.contains("|".join(target_cities), na=False) & ~df["Address"].isna() & ~df["Details"].str.contains("Residential", na=False)]
filtered_df = df

master_df = pd.read_csv("master.csv")
final_df = pd.concat([master_df, filtered_df], ignore_index=True, sort=True)
final_df = final_df.drop_duplicates(subset="Link", keep='first', inplace=False)

In [ ]:
from esri import get_df
for row in final_df.itertuples():
    if not(row.Inc1 > 0) or not(row.Pop1 > 0):
        try:
            esri_data = get_df(row.Address)
            
            final_df.loc[row.Index, ["Pop1"]] = esri_data[esri_data["Internal Name"] == "TOTPOP_CY"].iloc[0]["Ring 1"]
            final_df.loc[row.Index, ["Pop3"]] = esri_data[esri_data["Internal Name"] == "TOTPOP_CY"].iloc[0]["Ring 3"]
            final_df.loc[row.Index, ["Pop5"]] = esri_data[esri_data["Internal Name"] == "TOTPOP_CY"].iloc[0]["Ring 5"]

            final_df.loc[row.Index, ["Inc1"]] = esri_data[esri_data["Internal Name"] == "MEDHINC_CY"].iloc[0]["Ring 1"]
            final_df.loc[row.Index, ["Inc3"]] = esri_data[esri_data["Internal Name"] == "MEDHINC_CY"].iloc[0]["Ring 3"]
            final_df.loc[row.Index, ["Inc5"]] = esri_data[esri_data["Internal Name"] == "MEDHINC_CY"].iloc[0]["Ring 5"]
        except:
            continue

def apply_points(row):
    temp_points = 0
    if row["Pop1"] > 10000:
        temp_points += 1
    elif row["Pop1"] > 8000:
        temp_points += .5

    if row["Pop3"] > 35000:
        temp_points += 1
    elif row["Pop3"] > 28000:
        temp_points += .5

    if row["Inc1"] > 75000:
        temp_points += 1
    elif row["Inc1"] > 67500:
        temp_points += .5

    if row["Inc3"] > 75000:
        temp_points += 1
    elif row["Inc3"] > 67500:
        temp_points += .5
    
    return temp_points

def apply_hyperlink(row):
    return '=HYPERLINK("{}")'.format(row["Link"])

final_df["Points"] = final_df.apply(apply_points, axis=1)
final_df["Link_Click"] = final_df.apply(apply_hyperlink, axis=1)

In [ ]:
import shutil
shutil.move("master.csv", "Master/{}.csv".format(today.strftime("%Y-%m-%d %H.%M%p")))

final_df = final_df[["Link", "Address", "Price", "Size", "Details", "Upload", "Pop1", "Pop3", "Pop5", "Inc1", "Inc3", "Inc5", "Points", "Notes", "Link_Click"]]
final_df = final_df.sort_values(by=["Points"], ascending=[False])
final_df.to_csv("master.csv", index=False)